<center><img width="70%" src="logo.png"></center>

# **Projeto Observatório Social - Prefeitura Municipal de Indaial / SC**
### <font color='#DC143C'> *Webcrapping com Python | Criação e Alimentação de DW*
*by [JOVIANO SILVEIRA](https://www.joviano.com)*

---

# <font color=green> 1. BIBLIOTECAS E FUNÇÕES

In [1]:
# Importando bibliotecas

import time
import pandas as pd

from urllib.error import URLError, HTTPError
from urllib.request import Request, urlopen

import bs4


import warnings   
warnings.filterwarnings('ignore')

In [2]:
# Informando o AGENTE
# Para encontrá-lo, abra o site, aperte F12,Clique emNetwork/Clique no link do site/ no canto direito procure user-agent

agente = "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36"
headers = {'User-Agent': agente}

In [3]:
# Função para chamar a página WEB

def ConsultaWebB(url):
    try:
        req = Request(url, headers = headers)
        response = urlopen(req)
        return response.read()

    except:
        pass

In [4]:
# Função para pegar a página encontrada e transformar em um objeto utilizável

def captura_html_pagina(url):
    html = ConsultaWebB(url)
    soup = bs4.BeautifulSoup(html, 'html.parser')    
    return soup

In [5]:
# Função para capturar as informações da "tabelinha cabeçalho" da página e guardar numa variável dicionários 

def Cabecalho(html):
    dt = html.find_all('dt')
    dd = html.find_all('dd')
    dic = {}
    for i in range(len(dt)):
        x = dt[i].get_text()
        y = dd[i].get_text() 
        dic[x] = y
    return dic

In [6]:
# Funçao para capturar o conteúdo da página e transformar em um dicionário

def Conteudo(proposicao, ano):
    url = 'https://www.legislador.com.br//LegisladorWEB.ASP?WCI=ProposicaoTexto&ID=3&TPProposicao=1&nrProposicao='+str(proposicao)+'&aaProposicao='+str(ano)
    html = captura_html_pagina(url)
    dic = Cabecalho(html)
    dic['Proposição'] = proposicao
    dic['Ano'] = ano
    dic['Texto'] = html.p.get_text()
    return dic

In [7]:
def TabelaResultados(inicar_em, quantidade, ano, erros_admissiveis, segundos_espera):

    ultima_consulta = inicar_em + quantidade - 1

    # erros
    erros = 0

    # variaveis para loop
    i = 1
    lista = []

    while inicar_em <= ultima_consulta and erros <= erros_admissiveis:

        try:
            x = Conteudo(inicar_em,ano)
            lista = lista + [Conteudo(inicar_em,ano)]
        except:
            erros += 1
            pass

        time.sleep(segundos_espera)

        # carregamento incremental das variáveis
        inicar_em+=1 
        i+=1


    return pd.DataFrame(lista)

In [8]:
# Verificando se as informações estam sendo buscadas e adicionadas 
teste = TabelaResultados(500,3,2021,2,0.2)
teste

Reunião Deliberação             Situação  \
0  08/04/2021  08/04/2021  Proposição Aprovada   
1  08/04/2021  08/04/2021  Proposição Aprovada   
2  08/04/2021  08/04/2021  Proposição Aprovada   

                                             Assunto  \
0  Limpeza, Macadamização, Patrolamento, Retifica...   
1                      Pontes, Bueiros, boca de lobo   
2                          Manutenção de via pública   

                                     Autor  Proposição   Ano  \
0  Vereador Flávio Augusto Ferri Molinari.         500  2021   
1  Vereador Flávio Augusto Ferri Molinari.         501  2021   
2            Vereador Fernanda dos Santos.         502  2021   

                                               Texto  
0  O vereador que está subscreve, no uso das atri...  
1  O vereador que está subscreve, no uso das atri...  
2  A vereadora que esta subscreve, no uso das atr...

# <font color=green> 2. Listas

In [9]:
# Lista com nome de colunas
list(teste.columns)

['Reunião',
 'Deliberação',
 'Situação',
 'Assunto',
 'Autor',
 'Proposição',
 'Ano',
 'Texto']

In [10]:
# Lista com tipos de dados, interessante para verificar o tipo de dados 
# para criar a tabela
teste.dtypes

Reunião        object
Deliberação    object
Situação       object
Assunto        object
Autor          object
Proposição      int64
Ano             int64
Texto          object
dtype: object

# <font color=green> 3. BANCO DE DADOS E TABELAS

>Cria-se o banco de dados, deixar autenticação do Windows e criar a tabela

Para crie o banco de dados no SSMS, feito isso feche e abra o SSMS novamente para o SGBD reconhecer os campos criados.

Comando para criar Banco de dados e tabela

```sql
CREATE DATABASE Indaial2;

USE Indaial2;

CREATE TABLE Proposicoes (

	DataReuniao DATE,
	DataDeliberacao DATE,
	Situacao VARCHAR(200),
	Assunto VARCHAR(1000),
	Autor VARCHAR(1000),
	Proposicao INT,
	Ano INT,
	Texto VARCHAR(MAX)

)
```

# <font color=green> 4. ALIMENTANDO AS TABELAS

In [11]:
# Biblioteca que conecta o Python ao SQL
import pyodbc

In [ ]:
# No cmd execute esse comando para instalar o pyodbc
pip install pyodbc==4.0.30

In [12]:
# verifique a versão
pyodbc.version


'4.0.30'

In [13]:
# Conectando o Python com o SQL Server

'''
# Com senha para autenticação no Windows
conn = pyodbc.connect('Trusted_Connection=yes', 
                      driver = '{ODBC Driver 17 for SQL Server}',
                      server = 'DESKTOP-76N06IM\SQLEXPRESS', 
                      database = 'Indaial2',
                      UID='sa',
                      PWD='SuaSenha')


'''

#Com autenticação do Windows
conn = pyodbc.connect('Trusted_Connection=yes', 
                      driver = '{ODBC Driver 17 for SQL Server}',
                      server = 'DESKTOP-76N06IM\SQLEXPRESS', 
                      database = 'Indaial2')


In [26]:
# Buscando os dados com SQL e pandas para gerar tabela
    
query = '''
    select 
        * 
    from Proposicoes
'''
sql_query = pd.read_sql_query(query,conn)
sql_query

DataReuniao DataDeliberacao             Situacao  \
0    1996-02-22      1996-02-22  Proposição Aprovada   
1    1996-02-22      1996-02-22  Proposição Aprovada   
2    1996-02-22      1996-02-22  Proposição Aprovada   
3    1996-02-22      1996-02-22  Proposição Aprovada   
4    1996-02-22      1996-02-22  Proposição Aprovada   
..          ...             ...                  ...   
234  1996-03-04      1996-03-04  Proposição Aprovada   
235  1996-03-04      1996-03-04  Proposição Aprovada   
236  1996-03-04      1996-03-04  Proposição Aprovada   
237  1996-03-04      1996-03-04  Proposição Aprovada   
238  1996-03-04      1996-03-04  Proposição Aprovada   

                                               Assunto  \
0                                     Serviços e Obras   
1                                     Serviços e Obras   
2    Limpeza, Macadamização, Patrolamento, Retifica...   
3                      Rede de Água / Esgoto / Pluvial   
4                      Rede de Água / Esgoto / Pluvial   
..                                                 ...   
234                               Telefônia / Internet   
235      Iluminação Pública e Rede de Energia Elétrica   
236      Iluminação Pública e Rede de Energia Elétrica   
237                                           Diversos   
238                                           Diversos   

                        Autor  Proposicao   Ano  \
0    Vereador Henrique Fritz.           1  1996   
1    Vereador Henrique Fritz.           2  1996   
2    Vereador Henrique Fritz.           3  1996   
3    Vereador Henrique Fritz.           4  1996   
4    Vereador Henrique Fritz.           5  1996   
..                        ...         ...   ...   
234   Vereador Evaldir Simão.         235  1996   
235   Vereador Evaldir Simão.         236  1996   
236   Vereador Evaldir Simão.         237  1996   
237   Vereador Evaldir Simão.         238  1996   
238   Vereador Evaldir Simão.         239  1996   

                                                 Texto  
0    construção de Escola de 1º grau no Bairro Nova...  
1    construção de Escola nas imediações dos Loteam...  
2    alargamento da Rua ID 90                      ...  
3    prolongamento da rede d'água na Rua Reinhold S...  
4    Prolongamento de rede d'água na Rua Lorenz até...  
..                                                 ...  
234  Colocação de um telefone público na Rua Bagé, ...  
235  Verificar problemas de iluminação pública no p...  
236  Substituir lâmpadas queimadas, com a máxima ur...  
237  Efetuar reparos nos suportes para bicicletas l...  
238  Expedir aos servidores inativos contra cheque ...  

[239 rows x 8 columns]

In [14]:
# Criamos uma base
base = pd.DataFrame(columns=['Reunião', 'Deliberação', 'Situação', 'Assunto', 'Autor', 'Proposição', 'Ano', 'Texto'])
base

Empty DataFrame
Columns: [Reunião, Deliberação, Situação, Assunto, Autor, Proposição, Ano, Texto]
Index: []

In [ ]:
# "Combinamos" as bases

# essa prática é interessante para buscar todos os dados, mesmo que os campo estejam vazios
# usa-se 1900-01-01 para campos vazios(null)

base = base.append(teste)
base

In [ ]:
base.fillna('')

In [15]:
# append - faz a junção da base com os dados e o cabeçalho

base = pd.DataFrame(columns=['Reunião', 'Deliberação', 'Situação', 'Assunto', 'Autor', 'Proposição', 'Ano', 'Texto'])
teste = base.append(teste).fillna('')


# Conecta o banco de dados
conn = pyodbc.connect('Trusted_Connection=yes', 
                      driver = '{ODBC Driver 17 for SQL Server}',
                      server = 'DESKTOP-76N06IM\SQLEXPRESS', 
                      database = 'Indaial2')


# Inserindo os dados nas colunas
cursor = conn.cursor()


for index, row in teste.iterrows():
    
    cursor.execute('''
    
        INSERT INTO Proposicoes (
            DataReuniao,
            DataDeliberacao,
            Situacao,
            Assunto,
            Autor,
            Proposicao,
            Ano,
            Texto
        ) 
        values(?,?,?,?,?,?,?,?)''', # mesma quantidade de variáveis

        row['Reunião'], 
        row['Deliberação'], 
        row['Situação'], 
        row['Assunto'], 
        row['Autor'], 
        row['Proposição'], 
        row['Ano'], 
        row['Texto']

    )

# Executa o comando
conn.commit()
cursor.close()


In [ ]:
# Conexão com o Banco de dados
conn = pyodbc.connect('Trusted_Connection=yes', 
                      driver = '{ODBC Driver 17 for SQL Server}',
                      server = 'DESKTOP-76N06IM\SQLEXPRESS', 
                      database = 'Indaial2')

query = '''
    select 
        * 
    from Proposicoes
'''
sql_query = pd.read_sql_query(query,conn)
sql_query

In [16]:
# Funcão que conecta Python ao SQL com geração de tabela com os dados
def SQLSelect(query):
    
    conn = pyodbc.connect('Trusted_Connection=yes', 
                          driver = '{ODBC Driver 17 for SQL Server}',
                          server = 'DESKTOP-76N06IM\SQLEXPRESS', 
                          database = 'Indaial2')

    out = pd.read_sql_query(query,conn)
    return out

In [17]:
# Retorno da função acima
SQLSelect('''
    select 
        *
    from Proposicoes
''')

DataReuniao DataDeliberacao             Situacao  \
0  2021-04-08      2021-04-08  Proposição Aprovada   
1  2021-04-08      2021-04-08  Proposição Aprovada   
2  2021-04-08      2021-04-08  Proposição Aprovada   

                                             Assunto  \
0  Limpeza, Macadamização, Patrolamento, Retifica...   
1                      Pontes, Bueiros, boca de lobo   
2                          Manutenção de via pública   

                                     Autor  Proposicao   Ano  \
0  Vereador Flávio Augusto Ferri Molinari.         500  2021   
1  Vereador Flávio Augusto Ferri Molinari.         501  2021   
2            Vereador Fernanda dos Santos.         502  2021   

                                               Texto  
0  O vereador que está subscreve, no uso das atri...  
1  O vereador que está subscreve, no uso das atri...  
2  A vereadora que esta subscreve, no uso das atr...

In [18]:
SQLSelect('''
    select 
        maior = max(Proposicao)
    from Proposicoes
''')

maior
0    502

In [19]:
# Função que insere a próxima proposição que seja maior que a atual
# E envia para o Bando de Dados

def SQLInsertProposicoes(TabelaProposicoes):
    
    base = pd.DataFrame(columns=['Reunião', 'Deliberação', 'Situação', 'Assunto', 'Autor', 'Proposição', 'Ano', 'Texto'])
    TabelaProposicoes = base.append(TabelaProposicoes).fillna('')

    #conecta o banco de dados
    conn = pyodbc.connect('Trusted_Connection=yes', 
                          driver = '{ODBC Driver 17 for SQL Server}',
                          server = 'DESKTOP-76N06IM\SQLEXPRESS', 
                          database = 'Indaial2')

    cursor = conn.cursor()

    for index, row in TabelaProposicoes.iterrows():

        cursor.execute('''

            INSERT INTO Proposicoes (
                DataReuniao,
                DataDeliberacao,
                Situacao,
                Assunto,
                Autor,
                Proposicao,
                Ano,
                Texto
            ) 
            values(?,?,?,?,?,?,?,?)''', 

            row['Reunião'], 
            row['Deliberação'], 
            row['Situação'], 
            row['Assunto'], 
            row['Autor'], 
            row['Proposição'], 
            row['Ano'], 
            row['Texto']

        )

    conn.commit()
    cursor.close()

In [20]:
SQLInsertProposicoes(teste)

In [21]:
SQLSelect('''
    select 
        *
    from Proposicoes
''')

DataReuniao DataDeliberacao             Situacao  \
0  2021-04-08      2021-04-08  Proposição Aprovada   
1  2021-04-08      2021-04-08  Proposição Aprovada   
2  2021-04-08      2021-04-08  Proposição Aprovada   
3  2021-04-08      2021-04-08  Proposição Aprovada   
4  2021-04-08      2021-04-08  Proposição Aprovada   
5  2021-04-08      2021-04-08  Proposição Aprovada   

                                             Assunto  \
0  Limpeza, Macadamização, Patrolamento, Retifica...   
1                      Pontes, Bueiros, boca de lobo   
2                          Manutenção de via pública   
3  Limpeza, Macadamização, Patrolamento, Retifica...   
4                      Pontes, Bueiros, boca de lobo   
5                          Manutenção de via pública   

                                     Autor  Proposicao   Ano  \
0  Vereador Flávio Augusto Ferri Molinari.         500  2021   
1  Vereador Flávio Augusto Ferri Molinari.         501  2021   
2            Vereador Fernanda dos Santos.         502  2021   
3  Vereador Flávio Augusto Ferri Molinari.         500  2021   
4  Vereador Flávio Augusto Ferri Molinari.         501  2021   
5            Vereador Fernanda dos Santos.         502  2021   

                                               Texto  
0  O vereador que está subscreve, no uso das atri...  
1  O vereador que está subscreve, no uso das atri...  
2  A vereadora que esta subscreve, no uso das atr...  
3  O vereador que está subscreve, no uso das atri...  
4  O vereador que está subscreve, no uso das atri...  
5  A vereadora que esta subscreve, no uso das atr...

In [22]:
# Função para limpar dados da tabela

def SQLTruncate(NomeTabela):

    conn = pyodbc.connect('Trusted_Connection=yes', 
                          driver = '{ODBC Driver 17 for SQL Server}',
                          server = 'DESKTOP-76N06IM\SQLEXPRESS', 
                          database = 'Indaial2')

    cursor = conn.cursor()

    cursor.execute(f'''

                   TRUNCATE TABLE {NomeTabela}

                   ''')

    conn.commit()
    cursor.close()

In [23]:
# Limpando dados da tabela
SQLTruncate('Proposicoes')

# Visualizando a tabela
SQLSelect('select * from Proposicoes')

Empty DataFrame
Columns: [DataReuniao, DataDeliberacao, Situacao, Assunto, Autor, Proposicao, Ano, Texto]
Index: []

# <font color=green> 5. ALIMENTAÇÃO INCREMENTAL

In [24]:
# Limpar os dados da tabela só pra demonstração
# SQLTruncate('Proposicoes')


# ========= INICIO =========
proposicao = 202
ano = 2021
dados = Conteudo(proposicao,ano)
tabela = pd.DataFrame([dados])
SQLInsertProposicoes(tabela)
# ========= FIM =========


# visualizando dados
SQLSelect(f'select * from Proposicoes where Proposicao = {proposicao} and Ano = {ano}')

DataReuniao DataDeliberacao             Situacao  \
0  2021-02-11      2021-02-11  Proposição Aprovada   

                         Assunto                       Autor  Proposicao  \
0  Pontes, Bueiros, boca de lobo  Vereador Valentim Blasius.         202   

    Ano                                              Texto  
0  2021  O Vereador abaixo firmado requer, após ouvido ...

In [25]:
ano = 2021
dados = SQLSelect(f'select Proposicao = max(Proposicao) from Proposicoes where Ano = {ano}')
dados

Proposicao
0         202

In [26]:
# acessando a coluna
dados['Proposicao']

0    202
Name: Proposicao, dtype: int64

In [27]:
# acessando a coluna e linha
dados['Proposicao'].loc[0]

202

In [28]:
# ========= INICIO =========
ano = 2021

dados_ano = SQLSelect(f'select Proposicao = max(Proposicao) from Proposicoes where Ano = {ano}')
ultima_proposicao = dados_ano['Proposicao'].loc[0]

proxima_proposicao = int(ultima_proposicao) + 1

dados = Conteudo(proxima_proposicao,ano)
tabela = pd.DataFrame([dados])
SQLInsertProposicoes(tabela)
# ========= FIM =========


# visualizando dados
SQLSelect('select * from Proposicoes')

DataReuniao DataDeliberacao             Situacao  \
0  2021-02-11      2021-02-11  Proposição Aprovada   
1  2021-02-11      2021-02-11  Proposição Aprovada   

                                         Assunto                       Autor  \
0                  Pontes, Bueiros, boca de lobo  Vereador Valentim Blasius.   
1  Iluminação Pública e Rede de Energia Elétrica  Vereador Valentim Blasius.   

   Proposicao   Ano                                              Texto  
0         202  2021  O Vereador abaixo firmado requer, após ouvido ...  
1         203  2021  O Vereador abaixo firmado requer, após ouvido ...

In [29]:
# Limpando dados da tabela
SQLTruncate('Proposicoes')

# Visualizando a tabela
SQLSelect('select * from Proposicoes')

Empty DataFrame
Columns: [DataReuniao, DataDeliberacao, Situacao, Assunto, Autor, Proposicao, Ano, Texto]
Index: []

In [30]:
# Para acrescer a próxima proposição sem repetição
# Considerando que pode não ter nenhuma proposição ainda

# ========= INICIO =========
ano = 2021

dados_ano = SQLSelect(f'select Proposicao = max(Proposicao) from Proposicoes where Ano = {ano}')
ultima_proposicao = dados_ano['Proposicao'].loc[0]


# ----- INICIO ALTERAÇÃO ----- 
if ultima_proposicao == None:
    proxima_proposicao = 1
else:
    proxima_proposicao = int(ultima_proposicao) + 1 
# ----- FIM ALTERAÇÃO ----- 


dados = Conteudo(proxima_proposicao,ano)
tabela = pd.DataFrame([dados])
SQLInsertProposicoes(tabela)
# ========= FIM =========


# visualizando dados
SQLSelect('select * from Proposicoes')

DataReuniao DataDeliberacao             Situacao   Assunto  \
0  2021-02-01      2021-02-01  Proposição Aprovada  Trânsito   

                            Autor  Proposicao   Ano  \
0  Vereador Remir José de Faveri.           1  2021   

                                               Texto  
0  O vereador que esta subscreve, no uso das atri...

In [32]:
#Função para inserir dados

def InsereProximaProposicao (ano):

    # Busca última proposição cadastrada
    dados_ano = SQLSelect(f'select Proposicao = max(Proposicao) from Proposicoes where Ano = {ano}')
    ultima_proposicao = dados_ano['Proposicao'].loc[0]

    # Verifica se foi identificado lançamento naquele ano
    if ultima_proposicao == None:
        proxima_proposicao = 1
    else:
        proxima_proposicao = int(ultima_proposicao) + 1 

    # Captura e Insere dados na tabela
    dados = Conteudo(proxima_proposicao,ano)
    tabela = pd.DataFrame([dados])
    SQLInsertProposicoes(tabela)

In [33]:
InsereProximaProposicao(1996)
SQLSelect('select * from Proposicoes')

DataReuniao DataDeliberacao             Situacao           Assunto  \
0  2021-02-01      2021-02-01  Proposição Aprovada          Trânsito   
1  1996-02-22      1996-02-22  Proposição Aprovada  Serviços e Obras   

                            Autor  Proposicao   Ano  \
0  Vereador Remir José de Faveri.           1  2021   
1        Vereador Henrique Fritz.           1  1996   

                                               Texto  
0  O vereador que esta subscreve, no uso das atri...  
1  construção de Escola de 1º grau no Bairro Nova...

In [34]:
# Limpando dados da tabela
SQLTruncate('Proposicoes')

# Visualizando a tabela
SQLSelect('select *from Proposicoes')

Empty DataFrame
Columns: [DataReuniao, DataDeliberacao, Situacao, Assunto, Autor, Proposicao, Ano, Texto]
Index: []

In [35]:
ano_fim = 1999
ano_ini = 1996
anos = ano_fim - ano_ini + 1
for i in range(anos):
    print(i+ano_ini)

1996
1997
1998
1999


In [36]:
list(range(1996, 2000+1))

[1996, 1997, 1998, 1999, 2000]

In [ ]:
# Exemplo de Loop -que busca as proposições do ano 1996

ano = 1996
quantidade = 10
erros_admissiveis = 2
segundos_espera = 0.3

# erros
erros = 0

# variaveis para loop
i = 1
lista = []

while erros <= erros_admissiveis:

    try:
        InsereProximaProposicao(ano)
    except:
        erros += 1
        pass

    time.sleep(segundos_espera)

    # carregamento incremental das variáveis
    i +=1

In [37]:
# Limpando dados da tabela
SQLTruncate('Proposicoes')

# Visualizando a tabela
SQLSelect('select * from Proposicoes')

Empty DataFrame
Columns: [DataReuniao, DataDeliberacao, Situacao, Assunto, Autor, Proposicao, Ano, Texto]
Index: []

In [38]:
# Função que salva no banco e busca a próxima

def BuscaGravaDadosAno(ano, quantidade = 10, erros_admissiveis = 2, segundos_espera = 0.5):

    # erros
    erros = 0

    # variaveis para loop
    i = 1
    lista = []

    while erros <= erros_admissiveis:

        try:
            InsereProximaProposicao(ano)
        except:
            erros += 1
            pass

        time.sleep(segundos_espera)

        # carregamento incremental das variáveis
        i +=1

In [ ]:
ano_inicial = 1996
ano_final = 1997

for i in list(range(ano_inicial, ano_final+1)):
    print('Iniciando gravação dos dados do ano: ',i)
    try:
        BuscaGravaDadosAno(i, quantidade = 999999)
    except:
        pass
    
print('Inserção finalizada 😁')

Iniciando gravação dos dados do ano:  1996
